In [1]:
!nvidia-smi

Thu Jun 30 04:52:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
|  0%   37C    P8    23W / 370W |     43MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
|  0%   

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import os
from tableone import TableOne, load_dataset
import pandas as pd

 
print(tf.__version__)

warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')

2.5.0


In [3]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
copd_patient = pd.read_csv('Data/COPD_patient.csv')
copd_patient.head()

,subject_id
0,10000032
1,10000980
2,10001884
3,10002013
4,10002155


In [5]:
invalid_cxr = pd.read_csv('Data/Invalid_cxr.csv')
invalid_cxr

,icu_subject,study_id
0,10001884,"56722923,50712381,50376803,59305618"
1,10002428,"59891001,50444997,56597576,51574899,57506266,5..."
2,10003400,"54050001,52941169,55177950,50100991,50100991,5..."
3,10004235,"58604118,52604377,57318275,52962553"
4,10010867,"57317515,59036254,51983395,57260480,57907439,5..."
...,...,...
5911,19998843,"55683952,56350227,56350227,56773337,54376373"
5912,19999068,"54061983,58313972,59390811"
5913,19999287,"52519175,58938059,52519175"
5914,19999442,58708861


In [6]:
# demographic = pd.read_csv('Data/demographic.csv')
# import random
# from statistics import mode, StatisticsError

# def most_common(l):
#     if (len(l) == 1):
#         return l[0]
    
#     try:
#         most_common = mode(l)
#     except:
#         most_common = random.choice(l)
    
#     if (most_common == 3): 
#         second_common_list = [x for x in l if x != most_common]
#         if (len(second_common_list) > 0):
#             most_common = mode(second_common_list)
#     return most_common

# new_race_list = []
# for i in demographic['race']:
#     l = list(i)
#     l = [x for x in l if x != ',']
#     new_race_list.append(most_common(l))
    
# demographic.drop('race',axis=1, inplace=True)
# demographic['race'] = new_race_list

# demographic.to_csv('Data/Processed_demo.csv')

In [7]:
def get_vs_type(vs):
    if (len(vs) > 1):
        return 2
    elif (vs == 'Room air'):
        return 0
    elif (vs == 'HighFlow'):
        return 1
    else:
        return 2
    
def race_group(race):
    if (race > 4):
        return 3
    else:
        return race
    
def get_age_group(age):
    if (age < 40):
        return 0
    elif (age < 60):
        return 1
    elif (age < 80):
        return 2
    else:
        return 3

In [8]:
subject_ids = []
filename = ['TFrecords/mimic-tf-record{i}.tfrecords'.format(i=i) for i in range(24)]
# filename = ['lateral_tf-record{i}.tfrecords'.format(i=i) for i in range(4)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_ids.append(example.features.feature['subject_id'].int64_list.value[0]) # change subject_id to type int

In [9]:
train, test = train_test_split(np.unique(subject_ids), test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [10]:
# extract image, subject_id, study_id from tfrecords to 
demographic = pd.read_csv('Data/patient_age_gender.csv')

study_id_array = []
subject_id_array = []

img_train = []
subject_id_train = []
study_id_train = []
copd_train = []
disease_train = []
vs_train = []
age_train = []
races_train = []
gender_train = []

img_test = []
subject_id_test = []
study_id_test = []
copd_test = []
disease_test = []
vs_test = []
age_test = []
races_test = []
gender_test = []

img_val = []
subject_id_val = []
study_id_val = []
copd_val = []
disease_val = []
vs_val = []
age_val = []
races_val = []
gender_val = []

filename = ['../TFrecords/mimic-tf-record{i}.tfrecords'.format(i=i) for i in range(24)]
# filename = ['lateral_tf-record{i}.tfrecords'.format(i=i) for i in range(4)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    study_id = example.features.feature['study_id'].int64_list.value[0]
    
    invalid = invalid_cxr.loc[invalid_cxr['icu_subject']==subject_id, 'study_id'].values
    if (len(invalid) > 0 and str(study_id) in invalid[0].split(",")):
        continue
    
    if (subject_id in copd_patient['subject_id'].values):
        label = 1
    else:
        label = 0
                    
    sub_y = []
        
    sub_y.append(1 if example.features.feature['Atelectasis'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Cardiomegaly'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Consolidation'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Edema'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Enlarged Cardiomediastinum'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Fracture'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Lung Lesion'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Lung Opacity'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['No Finding'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pleural Effusion'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pleural Other'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pneumonia'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pneumothorax'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Support Devices'].float_list.value[0] == 1 else 0)
    
    race = race_group(example.features.feature['ethnicity'].int64_list.value[0])
    
    try:
        gender = 0 if demographic.loc[demographic['subject_id'] == subject_id, 'gender'].values[0] == 'F' else 1
    except:
        gender = 0
        
    try:
        age = demographic.loc[demographic['subject_id'] == subject_id, 'anchor_age'].values[0]
    except:
        age = 60
    
    byte_arr = example.features.feature['jpg_bytes'].bytes_list.value[0] # format image stored in Tfrecord
        
    if (subject_id in train):
        img_train.append(byte_arr) # img added to list 
        subject_id_train.append(subject_id) # subject_id add to list
        study_id_train.append(study_id)
        copd_train.append(label)
        gender_train.append(gender)
        age_train.append(age)
        races_train.append(race)
        disease_train.append(sub_y)
    elif (subject_id in val):
        img_val.append(byte_arr) # img added to list 
        subject_id_val.append(subject_id) # subject_id add to list
        study_id_val.append(study_id)
        copd_val.append(label)
        gender_val.append(gender)
        age_val.append(age)
        races_val.append(race)
        disease_val.append(sub_y)
    elif (subject_id in test):
        img_test.append(byte_arr) # img added to list 
        subject_id_test.append(subject_id) # subject_id add to list
        study_id_test.append(study_id)
        copd_test.append(label)
        gender_test.append(gender)
        age_test.append(age)
        races_test.append(race)
        disease_test.append(sub_y)
    else:
        continue
        
    subject_id_array.append(subject_id)    
    study_id_array.append(study_id)

In [14]:
# store data into a new tfrecord
np.random.seed(seed)

diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

record_file_train = 'copd_train_new.tfrecords' #output file 

with tf.io.TFRecordWriter(record_file_train) as writer: #write info into record_file 
    for i in range(len(img_train)):
        example = tf.train.Example() #create example and append img id and study_id to recrod file 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_train[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_train[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_train[i])

        example.features.feature['COPD'].int64_list.value.append(copd_train[i]) # labels 
        
#         example.features.feature['VS'].int64_list.value.append(vs_train[i])
        
        example.features.feature['race'].int64_list.value.append(races_train[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_train[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_train[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_train[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString()) #Write to a record file 
        
record_file_test = 'copd_val_new.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(img_val)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_val[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_val[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_val[i])

        example.features.feature['COPD'].int64_list.value.append(copd_val[i])
        
#         example.features.feature['VS'].int64_list.value.append(vs_val[i])
        
        example.features.feature['race'].int64_list.value.append(races_val[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_val[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_val[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_val[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString())

record_file_test = 'copd_test_new.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(img_test)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_test[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_test[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_test[i])

        example.features.feature['COPD'].int64_list.value.append(copd_test[i])
        
#         example.features.feature['VS'].int64_list.value.append(vs_test[i])
        
        example.features.feature['race'].int64_list.value.append(races_test[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_test[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_test[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_test[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString())

In [12]:
filename = ['copd_train_new.tfrecords', 'copd_test_new.tfrecords', 'copd_val_new.tfrecords']
# filename = ['lateral_tf-record{i}.tfrecords'.format(i=i) for i in range(4)]
raw_dataset = tf.data.TFRecordDataset(filename)
subject_id_array = []
copd_patient = []
for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id_array.append(example.features.feature['subject_id'].int64_list.value[0])
    
    if (example.features.feature['COPD'].int64_list.value[0] == 1):
        copd_patient.append(example.features.feature['subject_id'].int64_list.value[0])


# create a dataframe containing each patient uniquely

gender_array = []
age_array = []
races_array = []
copd_array = []
vs_array = []
miss = 0
for subject_id in np.unique(subject_id_array):    
    try:
        gender = 0 if demographic.loc[demographic['subject_id'] == subject_id, 'gender'].values[0] == 'F' else 1
        age = demographic.loc[demographic['subject_id'] == subject_id, 'anchor_age'].values[0]
        race = demographic.loc[demographic['subject_id'] == subject_id, 'race'].values[0]
    except:
        miss += 1
        gender = np.nan
        age = np.nan
        race = np.nan
        
    if (subject_id in copd_patient):
        label = 1
    else:
        label = 0

    copd_array.append(label)
    gender_array.append(gender)
    age_array.append(age)
    races_array.append(race)

df = pd.DataFrame({'subject_id':np.unique(subject_id_array), 'gender': gender_array, 
                              'age':age_array, 'race': races_array,
                              'COPD':copd_array
                             })
print(miss)
df.head()

112


,subject_id,gender,age,race,COPD
0,10000032,0.0,52.0,0.0,1
1,10000764,1.0,86.0,0.0,0
2,10000935,0.0,52.0,1.0,0
3,10000980,0.0,73.0,1.0,1
4,10001176,0.0,64.0,0.0,0


In [13]:
columns = ['gender', 'race', 'age']
categorical = ['gender', 'race']
groupby = 'COPD'
mytable = TableOne(df, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)

# 0: Female, 1: Male
# 0: White, 1: Black, 2: Latino, 3: others, 4: Asian
# 0: Rom air, 1: HighFlow, 2:Room air + HighFlow

mytable


Grouped by COPD                                                 
                           Missing       Overall             0            1 P-Value
n                                          51715         43763         7952        
gender, n (%)  0.0             112  26740 (51.8)  22671 (51.9)  4069 (51.2)   0.212
               1.0                  24863 (48.2)  20980 (48.1)  3883 (48.8)        
race, n (%)    0.0             112  33626 (65.2)  27815 (63.7)  5811 (73.1)  <0.001
               1.0                   8362 (16.2)   7260 (16.6)  1102 (13.9)        
               2.0                    3133 (6.1)    2828 (6.5)    305 (3.8)        
               3.0                    2592 (5.0)    2323 (5.3)    269 (3.4)        
               4.0                    1918 (3.7)    1745 (4.0)    173 (2.2)        
               5.0                    1669 (3.2)    1423 (3.3)    246 (3.1)        
               6.0                     175 (0.3)     149 (0.3)     26 (0.3)        
               7.0                     128 (0.2)     108 (0.2)     20 (0.3)        
age, mean (SD)                 112   59.9 (18.4)   58.4 (18.8)  67.9 (13.6)  <0.001